# Creates a  HDF5-Database for a measurement day  

In [13]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%autoreload 2

In [15]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
from collections import Counter
import pandas as pd
import numpy as np

'''Classes for Reading in the Data and applying DS Inversion and Interference corr'''
from nu_data_reduction import NU_data_read, normalisation, evaluation
from parameter import Sn_meas_obj, spike_obj
from dspike_formulas import *

import pylab as plt
from tables import *

# Define path and parameter of measurement

In [22]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2018-01-10/raw files/"
#path = "/Users/marf/Desktop/PhD Temp/2017-12-05/"
DS_inv_hdf = pd.HDFStore(path+"DS_inv_20171205.h5")

files_start = 9073
files_end = 9248

files_1 = range(files_start, files_end, 1)
#files_1 = range(8808,8915,1) + range(8922,9061, 1)
#files_1 = range(5552,5623,1) + range(5632,5729, 1)

In [17]:
# Isotopes used for Spike Inversion [[denominator isotope], nominator isotopes, .. ]
spikeSn_ls = [["117"],["118", "122", "124"]]

### spike and standard composition already defined in parameter.py ###

# Dictionary with pure DS composition
#spike_dict_117_122_meas = {"118": 0.0102951622334242, "119": 0.0090731830784630, "120" : 0.0338645175569080, "122" : 0.8068023572167630, "124" : 0.0114846216495240} # DS compositon 1ppm - 15.09.17

# Dictionary with Standard composition
#meas_dict = { "118" : 3.15777585757689, "119" : 1.11972979701600, "120" : 4.25050484928267, "122" : 0.60392336483548, "124" : 0.754692491283485} # mean of 1ppm NIST Std used for calib-15.09.17
# Denominator isotope in ratios for DS & Std composition
#meas_denom_117 = "117"

# Load compositions in for DS inversion
#Sn_meas_obj = load_ratio_dict(meas_dict, meas_denom_117)
#spike_obj = load_ratio_dict(spike_dict_117_122_meas, meas_denom_117)


In [18]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["117", "118", "119", "120", "122", "124"]
cup_config = cycles_spike

# Mass Range of cup configuration
mass_range = cycle_Sn_spike_mass_range

# Isotopes used for Interference correction
corr_isotopes_2 = {"Te" : "125", "Xe" : "129"}
#corr_isotopes_2 = {}
#corr_isotopes_Sb = {"Te": "125"}

denom_isotope = "117"
Sn_monitor = ["125"]

#def eval_iso_list(isotopes_list, denom_isotope, monitor_iso):
#    isotope_den = denom_isotope
#    isotopes_list.remove(isotope_den)
#    for i in range(len(monitor_iso)):
#        isotopes_list.append(monitor_iso[i])
#    isotopes_list.sort()
#   return isotopes_list

#isotope_ls = eval_iso_list(Sn_isotopes, denom_isotope, Sn_monitor)
  
isotopes = [["118", "119", "120", "122", "124"]]
data_sample_column = [(i + "/" + denom_isotope) for i in isotopes[0]]  

# columns for signal dataframe
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", 
             "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]

# Amount of interations for estimation of instrumental and natural fractionation
loop_nat = 3
loop_ins = 6
# assumended instrumental and natural fractionation
start_nat = -0.1 #start_nat = -0.1; start_nat = 1000
start_ins = 2 #start_ins = 2; start_ins = 0.001
# isotope ratio used for calculation of instrumental and natural fractionation [denominator[x, y, z]] e.g. [117,[118, 122, 124]] x = 118/117
inv_iso_ratio = 'x'

# Mass fractionation law ("exp" or "GPL")
law = "exp"

# n for GPL
n_GPL_ins = 0.3
n_GPL_nat = 0.001

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = True

# Methods to evaluate data & extract metadata

In [19]:
#def signals_raw(path, sample, cup_config, columns):
    
def metadata_df(data_read_obj, cycles):
    # extract sample name from csv
    sample_name = df.extract_metadata(sample, "Sample Name")
    # extract date from csv
    date = df.extract_metadata(sample, "Date")
    # extract time from csv
    starttime = df.extract_metadata(sample, "Start Time")
    # extract Number of Analysis Cycles from csv
    Num_of_Ana_Lines = df.extract_metadata(sample, "Number of Analysis Cycles")
    # combine date and starttime
    dateandtime = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    
    df_metadata = pd.DataFrame()
    df_metadata["Cycle"] = cycles
    df_metadata["Filenumber"] = sample
    df_metadata["Sample"] = sample_name
    df_metadata["Date"] = dateandtime
    df_metadata["Num_of_Ana_Cyc"] = Num_of_Ana_Lines
    df_metadata = df_metadata[["Filenumber", "Sample", "Date", "Cycle",  "Num_of_Ana_Cyc"]]
    df_metadata = df_metadata.set_index("Cycle")
    
    return df_metadata

def signals_raw(data_read_obj, sample, cup_config, cycle="cycle1"):
    # create a dataframe with raw signals from csv datafile
    signals_raw_all = df.data_read(sample)
    signals_raw = pd.DataFrame()
    for key in cup_config[cycle]:
        if key in signals_raw_all.columns:
            signals_raw[cup_config[cycle][key]] = signals_raw_all[key]
    
    signals_raw = signals_raw[sorted(signals_raw.columns)]
    return signals_raw

def signals_baseline(data_read_obj, sample, cup_config, cycle="zero1"):
    # create a dataframe with raw signals from csv datafile
    signals_base_all = df.data_read(sample)
    signals_base = pd.DataFrame()
    for key in cup_config[cycle]:
        if key in signals_base_all.columns:
            signals_base[cup_config[cycle][key]] = signals_base_all[key]
    
    signals_base = signals_base[sorted(signals_base.columns)]
    return signals_base

def signals_baseline_corr(data_read_obj, sample, cup_config, cycle="cycle1"):
    # creates a dataframe with all zero corrected signals on the cups

    ## perform baseline correction
    df_zero = df.data_zero_corr(sample)
    # create dataframe with baseline corrected data
    df_zero_1 = pd.DataFrame(df_zero[cycle])
    for key in cup_config[cycle]:
        if key in df_zero_1.columns:
            df_zero_1.rename(columns = {key : cup_config[cycle][key]}, inplace=True)
    
    df_zero_1 = df_zero_1[sorted(df_zero_1.columns)]
    df_zero_1.index.name = "Cycle"
    return df_zero_1
    
def bgd_before(data_read_obj, blk_ls, path, sample, cup_config):

    cycles = range(1, len(data_read_obj.data_read(sample).index)+1)
        
    # arbitrary blank positions - check blk position before and after sample 
    blk1 = [item for item in blk_ls if item < sample]
    
    if not blk1:
        return None
    else:
        blk1 = blk1[-1]
        # Reads signals of Blank measurements
        blk_1_obj = NU_data_read(path, blk1, cup_config)
        # Substracts baselines of signals
        df_bgd_1 = blk_1_obj.data_zero_corr(blk1)  

        return df_bgd_1

def bgd_after(data_read_obj, blk_ls, path, sample, cup_config):

    cycles = range(1, len(data_read_obj.data_read(sample).index)+1) 
    
    # arbitrary blank positions - check blk position before and after sample 
    blk2 = [item for item in blk_ls if item > sample]
    
    if not blk2:
        return None
    else:
        blk2 = blk2[0]
        # Reads signals of Blank measurements
        blk_2_obj = NU_data_read(path, blk2, cup_config)
        # Substracts baselines of signals
        df_bgd_2 = blk_2_obj.data_zero_corr(blk2)   
    
        return df_bgd_2

def raw_ratios(data_eval_obj):
    #Get raw ratios
    data_sample = data_eval_obj.raw_ratios(denom_isotope)
    # raw ratios from dataframe in dictionary for data handling
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')

    return data_sample

def df_plus_metadata(df_meta, df_values):
    df_values = pd.concat([df_meta, df_values], axis=1, ignore_index=False).reset_index()
    df_values = df_values.set_index("Date")
    
    return df_values

def add_epsilon_Q(df_values):
    #Calculation of eSn, fsam, fspike, Q
    
    
    df_values["eSn_118_117"] = ((df_values["Nr2:x"]/df_values["n0:x"])-1)*10000
    df_values["eSn_122_117"] = ((df_values["Nr2:y"]/df_values["n0:y"])-1)*10000
    df_values["eSn_124_117"] = ((df_values["Nr2:z"]/df_values["n0:z"])-1)*10000
    df_values["eSn_122_118"] = (((df_values["Nr2:y"]/df_values["Nr2:x"])/(df_values["n0:y"]/df_values["n0:x"]))-1)*10000
    df_values["eSn_124_118"] = (((df_values["Nr2:z"]/df_values["Nr2:x"])/(df_values["n0:z"]/df_values["n0:x"]))-1)*10000
    df_values["fsam"] = 1/((df_values["Mr2.5:x"]*0.07669971 - 0.24220048)/(0.00525699 - df_values["Mr2.5:x"]*0.51062751)+1)
    df_values["fspike"] = 1-df_values["fsam"]
    df_values["Q"] = df_values["fsam"]/df_values["fspike"]
    
    return df_values

# Database files for raw data

In [23]:
# Create Database for raw data

blk_ls = []
sample_ls = []

for sample in files_1:
    # read data from csv file
    df = NU_data_read(path, sample, cup_config)
    sample_name = df.extract_metadata(sample, "Sample Name")

    # determine number of cycles of file
    cycles = range(1, len(df.data_read(sample).index)+1)

    df_meta = metadata_df(df, cycles)

    ### baseline corrected signals
    df_signals = signals_raw(df, sample, cup_config)
    df_signals = df_plus_metadata(df_meta, df_signals)
    DS_inv_hdf.append('/raw_data/signals_raw', df_signals, min_itemsize = {"Sample": 30})
    
    df_baseline = signals_baseline(df, sample, cup_config)
    df_baseline = df_plus_metadata(df_meta, df_baseline)
    DS_inv_hdf.append('/raw_data/baselines', df_baseline, min_itemsize = {"Sample": 30})
    
    df_signals_zero = signals_baseline_corr(df, sample, cup_config)
    df_signals_zero = df_plus_metadata(df_meta, df_signals_zero)
    DS_inv_hdf.append('/raw_data/signals_zero', df_signals_zero, min_itemsize = {"Sample": 30})
    
    
    # create list with sample file numbers of sample measurements and blank measurements
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

# Database for DS_inv 

In [24]:
# Dictionary to loop over for different interference corrections
interf_corr_dict = {"No_interference_corr" : {}, "Te125_Xe129_corr" : {"Te" : "125", "Xe" : "129"}, "Te126_Xe129_corr" : {"Te" : "126", "Xe" : "129"}}

# List to loop over for bgd_corr and no_bgd_cor
bgd_corr_ls = [True, False]

# Outlier rejection un-/activated for background data
bgd_outlier_rej = True

for sample in sample_ls:
    
    # read data from csv file
    data_raw_obj = NU_data_read(path, sample, cup_config)
    # determine number of cycles of file
    cycles = range(1, len(data_raw_obj.data_read(sample).index)+1)
    # baseline corrected dataframe
    df_zero = data_raw_obj.data_zero_corr(sample)
    # dataframe with metadata(e.g. Filenumber, Date, etc.)
    df_meta = metadata_df(data_raw_obj, cycles)

    for bgd_corr in bgd_corr_ls:
        # iterate over type of interference correction
          
        for key in interf_corr_dict:

            # create Data_evaluation object
            data_eval_obj = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, interf_corr_dict[key], denom_corr_ratio, law, n_GPL_ins)

            # perform on peak background correction
            if bgd_corr == True:
                bgd1 = bgd_before(df, blk_ls, path, sample, cup_config)
                bgd2 = bgd_after(df, blk_ls, path, sample, cup_config)
                if bgd_outlier_rej == True:
                    bgd1 = data_eval_obj.mad_outlier_rejection_dict(bgd1)
                    bgd2 = data_eval_obj.mad_outlier_rejection_dict(bgd2)
                    bgd_method = 'bgd_outlier_corr'
                else:
                    bgd_method = 'bgd_no_outlier_corr'
                
                df_bgd_corr = pd.DataFrame(df_zero['cycle1']) - pd.DataFrame(data_eval_obj.data_bgd_corr_2(bgd1, bgd2)['cycle1'])
                for cup in cup_config['cycle1']:
                    if cup in df_bgd_corr.columns:
                        df_bgd_corr.rename(columns = {cup : cup_config['cycle1'][cup]}, inplace=True)
    
                df_bgd_corr = df_bgd_corr[sorted(df_bgd_corr.columns)]
            else:
                df_bgd_corr = pd.DataFrame()
                bgd_method = 'no_bgd_corr'
                
            # get raw ratios for DS-inversion
            df_raw_ratios = raw_ratios(data_eval_obj)

            # First DS-inversion without interference correction
            data_spike_obj_1 = calc_dspike_sample(Sn_meas_obj, df_raw_ratios, spike_obj, Sn_masses, spikeSn_ls , denom_isotope, law, n_GPL_ins, n_GPL_nat)
            data_spike_calc_1 = data_spike_obj_1.dspike_corr(loop_nat, loop_ins, start_nat, start_ins, inv_iso_ratio)
      
            # Beta for interference correction
            beta = data_spike_calc_1["frac_ins_x2.5"]

            # Interference correction
            spike_corr = data_eval_obj.norm_beta_to_raw("Sn", spikeSn_ls[0][0], beta)

            # interference corrected raw ratios
            df_corr_raw_ratios = pd.DataFrame.from_dict(spike_corr, orient = 'index')
    
            # amount of Interference correction
            df_inter_corr = df_raw_ratios - df_corr_raw_ratios
            df_raw_ratios = df_raw_ratios[sorted(df_raw_ratios.columns)]
            df_raw_ratios.columns = data_sample_column
            df_inter_corr = df_inter_corr[sorted(df_inter_corr.columns)]
            df_inter_corr.columns = data_sample_column
            
            # Second DS-inversion with interference corr raw_ratios
            data_spike_obj_2 = calc_dspike_sample(Sn_meas_obj, df_corr_raw_ratios, spike_obj, Sn_masses, spikeSn_ls , denom_isotope, law, n_GPL_ins, n_GPL_nat)
            data_spike_calc_2 = data_spike_obj_2.dspike_corr(loop_nat, loop_ins, start_nat, start_ins, inv_iso_ratio)

            df_raw_ratios = df_plus_metadata(df_meta, df_raw_ratios)
            
            df_corr_raw_ratios = df_plus_metadata(df_meta, df_corr_raw_ratios)
            DS_inv_hdf.append('/evaluation/'+bgd_method +'/' +key +'/df_raw_ratios_af_interf_corr', df_corr_raw_ratios, min_itemsize = {"Sample": 50})
            
            data_spike_calc_1 = df_plus_metadata(metadata_df(data_raw_obj, data_spike_calc_1.index.values), data_spike_calc_1)
            data_spike_calc_1["Cycle"] = data_spike_calc_1["Cycle"] + 1
            data_spike_calc_1 = add_epsilon_Q(data_spike_calc_1)
            DS_inv_hdf.append('/evaluation/'+bgd_method +'/' +key +'/df_DS_inv_bf_interf_corr', data_spike_calc_1, min_itemsize = {"Sample": 50})
            
            df_inter_corr = df_plus_metadata(df_meta, df_inter_corr) 
            DS_inv_hdf.append('/evaluation/'+bgd_method +'/' +key +'/df_amount_interf_corr', df_inter_corr, min_itemsize = {"Sample": 50})
            
            data_spike_calc_2 = df_plus_metadata(metadata_df(data_raw_obj, data_spike_calc_2.index.values), data_spike_calc_2)
            data_spike_calc_2["Cycle"] = data_spike_calc_2["Cycle"] + 1
            data_spike_calc_2 = add_epsilon_Q(data_spike_calc_2)
            DS_inv_hdf.append('/evaluation/'+bgd_method +'/' +key +'/df_DS_inv_af_interf_corr', data_spike_calc_2, min_itemsize = {"Sample": 50})

        DS_inv_hdf.append('/evaluation/'+bgd_method +'/df_raw_ratios', df_raw_ratios, min_itemsize = {"Sample": 50})
        df_bgd_corr = df_plus_metadata(df_meta, df_bgd_corr)
        DS_inv_hdf.append('/evaluation/'+bgd_method +'/df_bgd_corr' , df_bgd_corr, min_itemsize = {"Sample": 50})

# Close Database after adding all Data

In [21]:
DS_inv_hdf.close()

In [384]:
DS_inv_hdf.close()